In [1]:
from apiclient.discovery import build
import google_auth_httplib2
from google_auth_oauthlib.flow import InstalledAppFlow
import html
import TracklistParser
import YouTubeSearchEngine
import time
from FetchMP3 import getmp3
from SpotifyIntegration import SpotifyConnector

# Global Declaration
globalTracklistName = 0
client_secret_file = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube']  # Auth Endpoint for Youtube


def OAuthVerification():
    flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
    token = flow.run_console()
    youtube = build('youtube', 'v3', credentials=token)
    return youtube  # Returning Youtube Object


# Insert Playlist
def insertPlaylist(youtube, tracklistname):
    inserted_playlist_item = youtube.playlists().insert(part="snippet,status", body={
              "snippet": {
                "title": tracklistname,
                "description": tracklistname
              },
              "status": {
                "privacyStatus": "public"
              }
            })
    details = inserted_playlist_item.execute()
    # print(details['id'])
    playlist_id = details['id']
    return playlist_id


# Search for a Video element
def searchUsingAPI(youtube, trackTitle):
        searchResult = youtube.search().list(part='snippet', q=trackTitle, type='video', maxResults=20).execute()
        videoTitles = {}
        for snippet in searchResult['items']:
            videoId = snippet['id']['videoId']
            vidTitle = snippet['snippet']['title']
            videoTitles[videoId] = html.unescape(vidTitle)
        print(videoTitles)
                
        # -----------Video Compare Logic----------------
        keys = videoTitles.keys()
        matchResults = {}
        for key in keys:
            print(videoTitles[key])
            if str(videoTitles[key]).lower() == html.unescape(trackTitle).lower():
                matchResults[key] = 1
            else:
                matchResults[key] = 0
        print(matchResults)
         
        # -----------------Actual Video Id-----------------
        relativeVideoIds = []  # Which varies based on video title
        actualVideoIds = []  # List of videos which are exact match with the title
        matchResultsKeys = matchResults.keys()
        for eachKey in matchResultsKeys:
            if int(matchResults[eachKey]) == 1:
                actualVideoIds.append(eachKey)
            else:
                relativeVideoIds.append(eachKey)
        print("Relative Videos", relativeVideoIds)
        print("Actual Videos", actualVideoIds)
        
        # -----------------Adding Videos to Playlist------------
        if actualVideoIds:
            actualVideoId = actualVideoIds[0]
            print("Getting Actual Video", actualVideoId)
            return actualVideoId
        elif relativeVideoIds:
            actualVideoId = relativeVideoIds[0]
            print("Getting Relative Video", actualVideoId)
            return actualVideoId
        else:
            return "Video Not Found"


def insertTracks(youtube, playlistId, cleanedTracks):  # youtube object is required only for Searching via API
    print("Creating your Youtube Playlist. It may take a while....")
    playListData = {}
    for trackTitle in cleanedTracks:
        time.sleep(3)
        # Search using---------------------- Youtube API
        # actualVideoId = searchUsingAPI(youtube,trackTitle)
        # Search Using---------------------- Youtube Crawler
        actualVideoId = YouTubeSearchEngine.searchUsingScraper(trackTitle)
        if actualVideoId == "Video Not Found":
            print(actualVideoId)
        elif actualVideoId == "Failed To Generate URL":
            print(actualVideoId)
        else:
            print()
            print(trackTitle)
            playListData[actualVideoId] = trackTitle
            insertPlaylistItem(playlistId, actualVideoId)
    return playListData


def insertPlaylistItem(playlistId, actualVideoId):
    youtube.playlistItems().insert(part='snippet', body={
      "snippet": {
        "playlistId": playlistId,
        "resourceId": {
          "kind": "youtube#video",
          "videoId": actualVideoId
        }
      }
    }).execute()


if __name__ == '__main__':  # Program initiation point
    flg = True
    while flg:
        tracklistCleanedName, cleanedTracks = TracklistGenerator.generateTracklist()
        service = int(input('Please select Service to save Tracks: 1.Youtube 2.Spotify ? [1 or 2]: '))
        if service == 1:
            youtube = OAuthVerification()
            playlistId = insertPlaylist(youtube, tracklistCleanedName)
            playListData = insertTracks(youtube, playlistId, cleanedTracks)
            prompt = str(input("Do You Want to Download the Tracks Locally ? [Yes/No]: ")).lower()
            if prompt == 'yes' or prompt == 'y' and playListData:
                getmp3(playListData)
                print("Tracks Downloaded")
            elif not playListData:
                print("PlayList is empty")
            else:
                print("Tracks Successfully Added on Youtube")
        elif service == 2:
            if SpotifyConnector(tracklistCleanedName, cleanedTracks):  # Returns a Boolean
                print("Successfully created a Spotify Playlist")
            else:
                print("An Error Occurred while creating a Spotify Playlist")
        end_choice = input('Do You Want To Add More Tracks Mate !? [Yes/No]')
        if end_choice.lower() == 'yes':
            flg = True
        else:
            flg = False
            

Enter Tracklist URL : https://www.mixesdb.com/w/2019-09-23_-_Motez_-_Big_Beat_Radio_EP_065_(Music_With_Purpose_Mix)

Fetching Tracklist.....

Tracks To Add : 11
Cowboy Rhythmbox - Terminal Madness
Nozz - Lost Memories II
Pardon Moi - Power To The People (Damon Jee Remix)
Krystal Klear - Miyoki
Motez - Toggle
Underground System - Just A Place (Gerd Janson Remix)
Jansons Feat. Dope Earth Alien - Switch
Motez - Where Have You Been
My Cat Snoop - Don t Ya Know
Dino Lenny - Tokyo (Black Mami Remix)
CamelPhat & Jake Bugg - Be Someone (Cristoph Remix)
Please select Service to save Tracks: 1.Youtube 2.Spotify ? [1 or 2]: 2
You will be redirected to Google page. Please enter the Access Token below.

Enter the Access Token : BQCEE4WtTc4AudR9rnC9aAO6JLwnL8RknlBEw0Daq9eU25uwT4KOV1U91nEHGNZ7gPHIvy5FkUyt8OB3z33Y9tGKZotF3d5ESotOllR6HSCWFIc2DPJLUP8k47v1UBtwfX8Ybh78ZY_i-9T9cFITKhQ5b4BjQMn3eG7Vhcb-eo6Ek-QGdzTe0mQF4huC_MyA-EEAY2LrmsGsKgGagnrGbjqjh9T3nsxxj8vWNpURg4-3CGn5VCF1D9BUIw&token_type=Bearer&expire